In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize
from requests.auth import HTTPBasicAuth

from dotenv.main import load_dotenv
import os

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import nltk
from nltk.tokenize import word_tokenize

import math

from sklearn.preprocessing import StandardScaler

import pickle
import networkx as nx
from collections import defaultdict
import itertools


pd.options.display.max_seq_items = 20000
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 2000)

from dotenv.main import load_dotenv
import os
load_dotenv()
API_KEY=os.environ["api_key"]

In [2]:
def level1_extractor_list(x,field):
    try:
        a=pd.json_normalize(x)[field].tolist()
    except:
        a=[]
    return a

def level1_extractor_dict(x,key,value):
    try:
        a=dict(zip(pd.json_normalize(x)[key],pd.json_normalize(x)[value]))
    except:
        a={}
    return a

def level1_extractor_max(x,field):
    try:
        a=max(list(pd.json_normalize(x)[field]))
    except:
        a=[]
    return a

def level2_extractor_list(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=list([c for b in list(mp) if len(b)!=0 for c in b])
    except:
        a=[]
    return a

def level2_extractor_max(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=max([x for x in list(mp) if x!=None])
    except:
        a=[]
    return a

def level2_extractor_min(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=min([x for x in list(mp) if x!=None])
    except:
        a=[]
    return a

def level2_extractor_avg(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=np.mean([x for x in list(mp) if x!=None])
    except:
        a=[]
    return a

In [5]:
api_url="https://api.dealroom.co/api/v1/founders/filters"
auth=HTTPBasicAuth(API_KEY, '')

headers = {'Content-Type': 'application/json'}
data={"fields":"locations",
              "limit": 1,
              "offset": 0
             }
response = requests.post(api_url,data=json.dumps(data),auth=auth,headers=headers)
data=response.json()


In [4]:
df=pickle.load(open('/Users/klaus/Documents/Jarvis/Second Try/app/model/NetworkAnal.ipynb','rb'))

UnpicklingError: invalid load key, '{'.

In [3]:
import time
import requests
from requests.auth import HTTPBasicAuth
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [4]:
fields="id,name,deleted,path,tagline,about,investor_type,sub_types,url,website_url,twitter_url,facebook_url,linkedin_url,google_url,crunchbase_url,angellist_url,playmarket_app_id,appstore_app_id,images,investment_stages,employees,employees_latest,hq_locations,deal_size,launch_year,launch_month,industry_experience,country_experience,client_focus,tags,deal_structure,total_funding,total_funding_source,recent_funding,recent_funding_source,last_funding,last_funding_source,investments_num,top_investments_num,dutch_investments_num,last_updated,last_updated_utc,created_utc,europe_seed_prominence_unique,europe_seed_prominence_percentile,europe_seriesa_prominence_unique,europe_seriesa_prominence_percentile,emea_seed_prominence_2021_unique,emea_seed_prominence_2021_percentile,global_seed_prominence_2021_unique,global_seed_prominence_2021_percentile,emea_seriesa_prominence_2021_unique,emea_seriesa_prominence_2021_percentile,global_seriesa_prominence_2021_unique,global_seriesa_prominence_2021_percentile,multistage_emea_prominence_2021_unique,multistage_emea_prominence_2021_percentile,multistage_global_prominence_2021_unique,multistage_global_prominence_2021_percentile,emea_seed_prominence_2022_unique,emea_seed_prominence_2022_percentile,global_seed_prominence_2022_unique,global_seed_prominence_2022_percentile,emea_seriesa_prominence_2022_unique,emea_seriesa_prominence_2022_percentile,global_seriesa_prominence_2022_unique,global_seriesa_prominence_2022_percentile,multistage_emea_prominence_2022_unique,multistage_emea_prominence_2022_percentile,multistage_global_prominence_2022_unique,multistage_global_prominence_2022_percentile,accelerator_prominence_2021_unique,accelerator_prominence_2021_percentile,team,fundings,investments,co_investors,funds_under_management"
api_url="https://api.dealroom.co/api/v1"
api_key = API_KEY  # Replace with the issued Dealroom API key
AUTH = HTTPBasicAuth(api_key, "")
HEADERS = {'Content-Type': 'application/json'}

json_list=[]
k=0

# Set the parameters depending on the use case. See refs:
# https://urllib3.readthedocs.io/en/latest/reference/urllib3.util.html#module-urllib3.util.retry
retry_strategy = Retry(
    backoff_factor=1,
    total=3,
    read=3,
    status_forcelist=[500, 502, 503, 504],
    method_whitelist=False, # This is important to force retry of POST requests
)
ADAPTER = HTTPAdapter(max_retries=retry_strategy)

def get_session() -> requests.Session:
    """Return a Session object initialized with correct parameters."""

    session = requests.Session()
    session.headers = HEADERS
    session.auth = AUTH
    session.mount(api_url, ADAPTER)
    return session


def post_request(
        session: requests.Session,
        url: str,
        data: dict
    ) -> requests.models.Response:
    """Make a POST request using input Session object. If ConnectionError is
    raised, we refresh the session."""

    try:
        return session.post(url, json=data)

    except requests.exceptions.ConnectionError as exc:
        exc_str = f"{exc.__class__.__name__}: {exc}"
        print(f"POST request raised: {exc_str}. Refreshing session.")
        session = get_session()
        return session.post(url, json=data)


def get_response_body(response: requests.models.Response) -> dict:
    """Get response body. If JSONDecodeError is raised, returns an empty
    dictionary."""

    try:
        return response.json()

    except json.JSONDecodeError as exc:
        exc_str = f"{exc.__class__.__name__}: {exc}"
        print(f"Getting response body raised: {exc_str}. Returning empty dictionary.")
        return {}

                
            

/var/folders/k0/ldv3tm456fl7jrgd0xn0vy5h0000gn/T/ipykernel_5731/904110707.py:12: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  retry_strategy = Retry(


In [7]:
json_list=[]
if __name__ == "__main__":

    session = get_session()
    url = f"{api_url}/investors/bulk"

    next_page_id = ''
    k=0

    while True:
        data = {"form_data": {"must": {"country_experience": ["Europe"]}},
            "fields": fields,
            'next_page_id': next_page_id,
            'limit': 100,
            }
        
        r = post_request(session, url, data)
        res = get_response_body(r)
        json_list.append(res)
        k+=1
        if (k/10).is_integer():
            if k/10==1:
                print("")

            else: print([k,next_page_id],end="\r")
        try:
            next_page_id = res['next_page_id']
        except:
            print(k,next_page_id)
        if next_page_id is None:
            break


NameError: name 'get_session' is not defined

Missing Data Function

In [3]:
fields="id,name,deleted,path,tagline,about,investor_type,sub_types,url,website_url,twitter_url,facebook_url,linkedin_url,google_url,crunchbase_url,angellist_url,playmarket_app_id,appstore_app_id,images,investment_stages,employees,employees_latest,hq_locations,deal_size,launch_year,launch_month,industry_experience,country_experience,client_focus,tags,deal_structure,total_funding,total_funding_source,recent_funding,recent_funding_source,last_funding,last_funding_source,investments_num,top_investments_num,dutch_investments_num,last_updated,last_updated_utc,created_utc,europe_seed_prominence_unique,europe_seed_prominence_percentile,europe_seriesa_prominence_unique,europe_seriesa_prominence_percentile,emea_seed_prominence_2021_unique,emea_seed_prominence_2021_percentile,global_seed_prominence_2021_unique,global_seed_prominence_2021_percentile,emea_seriesa_prominence_2021_unique,emea_seriesa_prominence_2021_percentile,global_seriesa_prominence_2021_unique,global_seriesa_prominence_2021_percentile,multistage_emea_prominence_2021_unique,multistage_emea_prominence_2021_percentile,multistage_global_prominence_2021_unique,multistage_global_prominence_2021_percentile,emea_seed_prominence_2022_unique,emea_seed_prominence_2022_percentile,global_seed_prominence_2022_unique,global_seed_prominence_2022_percentile,emea_seriesa_prominence_2022_unique,emea_seriesa_prominence_2022_percentile,global_seriesa_prominence_2022_unique,global_seriesa_prominence_2022_percentile,multistage_emea_prominence_2022_unique,multistage_emea_prominence_2022_percentile,multistage_global_prominence_2022_unique,multistage_global_prominence_2022_percentile,accelerator_prominence_2021_unique,accelerator_prominence_2021_percentile,team,fundings,investments,co_investors,funds_under_management"

In [4]:
def missing_info(id,fields):
    l=[]
    k=0
    for i in id: 
        try:
            api_url="https://api.dealroom.co/api/v1/investors/batch?ids="+i+"&fields="+fields
            auth=HTTPBasicAuth(API_KEY, '')
            headers = {
                "Content-Type": "application/json",
                "Authorization": "Basic undefined"
            }

            response = requests.get(url=api_url, auth=auth, headers=headers)

            res=json.loads(response.text)
            l.append(res)
        
        except: None


    return print(api_url)

In [112]:
l=[]
k=0
for i in ids: 
    k+=1
    print(k,end="\r")
    try:
        api_url="https://api.dealroom.co/api/v1/investors/batch?ids="+str(i)+"&fields="+fields
        auth=HTTPBasicAuth(API_KEY, '')
        headers = {
            "Content-Type": "application/json",
            "Authorization": "Basic undefined"
        }

        response = requests.get(url=api_url, auth=auth, headers=headers)

        res=json.loads(response.text)
        l.append(res)
    
    except: None

In [110]:
ids=list(missing_investors.keys())

In [111]:
len(ids)

354

In [ ]:
api_url="https://api.dealroom.co/api/v1/investors/batch?ids="+ids+"&fields="+fields
auth=HTTPBasicAuth(API_KEY, '')
headers = {
    "Content-Type": "application/json",
    "Authorization": "Basic undefined"
}

response = requests.get(url=api_url, auth=auth, headers=headers)

res=json.loads(response.text)


In [41]:
with open("Investors.json", "w") as fp:
    json.dump(json_list, fp) 

In [5]:
with open("/Users/klaus/Documents/Jarvis/Second Try/app/model/Network Analysis/Inv./Investors.json", 'r') as fp:
    investor_json = json.load(fp)
with open("/Users/klaus/Documents/Jarvis/Second Try/app/model/Network Analysis/Inv./Eurpe_Founders.json", 'r') as fp:
    founders_json = json.load(fp)
with open("/Users/klaus/Documents/Jarvis/Second Try/app/model/Network Analysis/Inv./Universities.json", 'r') as fp:
    university_json = json.load(fp)
    

In [6]:
import uuid

def assign_unique_id(value):
    # Generate a UUID based on the SHA-1 hash of a namespace identifier and a name
    dict_id = uuid.uuid5(uuid.NAMESPACE_DNS, str(value)).int

    # Add the ID to the dictionary
    

    return dict_id


Needed cols:
1. name
2. investment stages
3. employees
4. hq_locations
5. deal size
6. launch year
7. industry experience
8. country experience
9. total funding
10. last funding
11. investments_num
12. european investments (country specific)
13. number of funds under management
14. co_investors
15. team members
16. total fundings
17. co_investors total

Available cols:


In [7]:
def jsonListTransformerFounder(apiOutput: list):
    networkInputList = []
    for singleJsonList in apiOutput: 
        jsonList = singleJsonList.get("items", [])
        for individualJson in jsonList:  
            if individualJson.get("linkedin_url"):     
                networkInput = {}
                networkInput["id"]=assign_unique_id(individualJson.get("name"))
                networkInput["dealroom_id"] = individualJson.get("id")
                networkInput["type"] = individualJson.get("type")
                networkInput["name"] = individualJson.get("name")
                networkInput["backgrounds"] = [x.get("name") for x in individualJson.get("backgrounds", []) if x.get("name")]
                networkInput["linkedin_url"] = individualJson.get("linkedin_url")
                networkInput["city"] = [x["city"].get("name") for x in individualJson.get("hq_locations", []) if x.get("city")]
                networkInput["universities"] = [{assign_unique_id(x["university"]["name"]): x["university"]["name"]} for x in individualJson.get("universities", {}).get("items", []) if x.get("university")]
                networkInput["universities_degree"] = [x["degree"].get("name") for x in individualJson.get("universities", {}).get("items", []) if x.get("degree")]
                networkInput["companies"] = [{assign_unique_id(x.get("name")):x.get("name")} for x in individualJson.get("companies", {}).get("items", []) if x.get("id")]
                networkInput["companies_titles"] = [sublist.get("name") for x in individualJson.get("companies", {}).get("items", []) for sublist in x.get("titles", []) if sublist.get("name")]
                networkInput["founded_companies"] = sum([x.get("is_founder", 0) for x in individualJson.get("companies", {}).get("items", [])])
                networkInput["founder_score"] = individualJson.get("founder_score")
                networkInput["is_strong_founder"] = individualJson.get("is_strong_founder")
                networkInput["is_super_founder"] = individualJson.get("is_super_founder")
                networkInput["is_promising_founder"] = individualJson.get("is_promising_founder")
                networkInput["founded_companies_total_funding"] = individualJson.get("founded_companies_total_funding")
                networkInput["universities_total"] = individualJson.get("universities", {}).get("total")
                networkInput["companies_total"] = individualJson.get("companies", {}).get("total")
                networkInputList.append(networkInput)
    return networkInputList


In [8]:
def jsonListTransformerCompany(apiOutput: list):
    networkInputList=[]
    for singleJsonList in apiOutput:
        jsonList = singleJsonList.get("items", [])
        for individualJson in jsonList:
            if individualJson.get("website_url") is not None:
                networkInput={}
                networkInput["id"]=assign_unique_id(individualJson.get("name"))
                networkInput["dealroom_id"]=individualJson.get("id")
                networkInput["name"]=individualJson.get("name")
                networkInput["about"]=individualJson.get("about")
                networkInput["tagline"]=individualJson.get("tagline")
                networkInput["website_url"]=individualJson.get("website_url")
                networkInput["industries"]=[x.get("name") for x in individualJson.get("industries", []) if x.get("name")]
                networkInput["city"]=[x["city"].get("name") for x in individualJson.get("hq_locations", []) if x.get("city")]
                networkInput["country"]=[x["country"].get("name") for x in individualJson.get("hq_locations", []) if x.get("country")]
                networkInput["total_funding"]=individualJson.get("total_funding_source")
                networkInput["company_status"]=individualJson.get("company_status")
                networkInput["ownerships"]=individualJson.get("ownerships")
                networkInput["patents_count"]=individualJson.get("patents_count")
                networkInput["launch_year"]=individualJson.get("launch_year")
                networkInput["investors_items"]=jsonListTransformerInvestors([individualJson.get("investors", {})]) if individualJson.get("investors", {}).get("total") != 0 else []
                networkInput["team"]=jsonListTransformerFounder([individualJson.get("team", {})]) if individualJson.get("team", {}).get("total") != 0 else []
                networkInput["fundings"]=[{"amount":x.get("amount_source"),"date":[x.get("month"),x.get("year")] } for x in individualJson.get("fundings", {}).get("items", []) if x.get("id")]
                networkInputList.append(networkInput)          
                
    return networkInputList

In [9]:
def jsonListTransformerInvestors(apiOutput: list):
    networkInputList=[]
    for singleJsonList in apiOutput:
        jsonList = singleJsonList.get("items", [])
        for individualJson in jsonList:
            if individualJson.get("website_url"):
                networkInput={}
                networkInput["id"]=assign_unique_id(individualJson.get("name"))
                networkInput["dealroom_id"]=individualJson.get("id")
                networkInput["name"]=individualJson.get("name")
                networkInput["about"]=individualJson.get("about")
                networkInput["investor_type"]=individualJson.get("investor_type")
                networkInput["website_url"]=individualJson.get("website_url")
                networkInput["linkedin_url"]=individualJson.get("linkedin_url")
                networkInput["team"]=jsonListTransformerFounder([individualJson.get("team", {})]) if individualJson.get("team", {}).get("total") != 0 else []
                networkInput["team_total"]=individualJson.get("team", {}).get("total")
                networkInput["city"]=[x["city"].get("name") for x in individualJson.get("hq_locations", []) if x.get("city")]
                networkInput["country"]=[x["country"].get("name") for x in individualJson.get("hq_locations", []) if x.get("country")]
                networkInput["deal_size"]=individualJson.get("deal_size")
                networkInput["launch_year"]=individualJson.get("launch_year")
                networkInput["launch_month"]=individualJson.get("launch_month")
                networkInput["industry_experience"]=[x.get("name") for x in individualJson.get("industry_experience", []) if x.get("name")]
                networkInput["country_experience"]=[x.get("name") for x in individualJson.get("country_experience", []) if x.get("name")]
                networkInput["tags"]=individualJson.get("tags")
                networkInput["deal_structure"]=individualJson.get("deal_structure")
                networkInput["total_funding"]=individualJson.get("total_funding_source")
                networkInput["last_funding_source"]=individualJson.get("last_funding_source")
                networkInput["investments"]=[{assign_unique_id(x.get("name")):x.get("name")} for x in individualJson.get("investments", {}).get("items", []) if x.get("id")]
                networkInput["investments_num"]=individualJson.get("investments_num")
                networkInput["top_investments_num"]=individualJson.get("top_investments_num")
                networkInput["fundings"]=networkInput["fundings"]=[
                                                                     {
                                                                    "amount": x.get("amount_source"),
                                                                    "valuation": x.get("valuation"),
                                                                    "date": [x.get("month"), x.get("year")],
                                                                    "investors": [{assign_unique_id(sublist.get("name")): sublist.get("name")} for sublist in x.get("investors", [])]
                                                                } for x in individualJson.get("fundings", {}).get("items", [])
                                                            ]

                networkInput["co_investors"]=[{assign_unique_id(x.get("name")):x.get("name")} for x in individualJson.get("co_investors", {}).get("items", []) if x.get("id")]
                networkInput["co_investors_total"]=individualJson.get("co_investors", {}).get("total")
                networkInput["funds_under_management"]=[{assign_unique_id(x.get("name")):x.get("name")} for x in individualJson.get("funds_under_management", {}).get("items", []) if x.get("id")]
                networkInput["funds_under_management_total"]=individualJson.get("funds_under_management", {}).get("total")
                networkInputList.append(networkInput)
    return networkInputList

                

In [10]:
def jsonListTransformerUniversity(apiOutput: list):
    networkInputList=[]
    for singleJsonList in apiOutput:
        jsonList = singleJsonList.get("items", [])
        for individualJson in jsonList:
            if individualJson.get("website_url"):
                networkInput={}
                networkInput["id"]=assign_unique_id(individualJson.get("name"))
                networkInput["id_dealroom"]=individualJson.get("id")
                networkInput["name"]=individualJson.get("name")
                networkInput["about"]=individualJson.get("about")
                networkInput["website_url"]=individualJson.get("website_url")
                networkInput["linkedin_url"]=individualJson.get("linkedin_url")
                networkInput["city"]=[x["city"].get("name") for x in individualJson.get("hq_locations", []) if x.get("city")]
                networkInput["country"]=[x["country"].get("name") for x in individualJson.get("hq_locations", []) if x.get("country")]
                networkInput["univ_companies"]=[{assign_unique_id(x.get("name")):x.get("name")} for x in individualJson.get("university_companies_items", []) if x.get("id")]
                networkInput["univ_alumni"]=[{assign_unique_id(x.get("name")):x.get("name")} for x in individualJson.get("university_alumni_items", []) if x.get("id")]
                networkInput["univ_companies_total"]=individualJson.get("university_companies_total")
                networkInput["university_founders_total"]=individualJson.get("university_founders_total")
                networkInput["university_founders_raised_10m_total"]=individualJson.get("university_founders_raised_10m_total")
                networkInput["university_founders_uni_companies_total"]=individualJson.get("university_founders_unicorn_companies_total")
                networkInputList.append(networkInput)
    return networkInputList

Data Preperation 

Goal is to look at the data for each network and see if any types of nodes appear which we dont want

Investors

In [11]:
investorJsonClean=jsonListTransformerInvestors(investor_json)

In [12]:
with open("Investors_clean.json", "w") as fp:
    json.dump(investorJsonClean, fp) 

In [16]:
import gzip
with gzip.open("investor_clean.json.gz", 'wt', encoding='utf-8') as f:
    json.dump(investorJsonClean, f)

In [ ]:
with gzip.open("data.json.gz", 'rt', encoding='utf-8') as f:
    data = json.load(f)

In [20]:
pd.unique([d["investor_type"] for d in investorJsonClean])

array(['government & non-profit', 'corporate', 'company',
       'investment fund', 'crowdfunding'], dtype=object)

In [23]:
investorJsonExtracted=[d for d in investorJsonClean if d["investor_type"] not in ["crowdfunding"]]

Build the network for investors

In [12]:
g_inv=nx.Graph()

In [111]:
for investor in investorJsonClean:
    attr=investor.copy()
    node_name=attr.pop("name")
    g_inv.add_node(node_name,**attr)

connect investors in the same city

In [134]:

# Create a mapping from cities to nodes
city_inv_dict = defaultdict(list)   
for node,data in g_inv.nodes(data=True):
    cities=data.get("city")
    for city in cities:
        if city is not None:
            city_inv_dict[city].append(node)




In [127]:


for key, values in city_inv_dict.items():
    print(key)
    input=list(itertools.product(values,values))
    seen = set()
    # using list comprehension
    Output = [(a, b, 2) for a, b in input if not (a == b) ]
    g_inv.add_weighted_edges_from(Output, edge_type="city", city=key, edge_color="red")
    

Belfast
Stuttgart
Esplugues de Llobregat
Barcelona
Palma
Lisbon
Beynes
Stockholms kommun
London
Zurich
Farmington Hills
Empoli
Barendrecht
Wakefield
Rosheim
Stadt Imst
Alpen
L'Hospitalet de Llobregat
Cesena
Thiruvananthapuram
Obertshausen
Almere
Brno
Almen
Heidelberg
Santa Cruz de Tenerife
Bilbao
Città di Castello
Columbus
Nijmegen
Doncaster
Meyrin
Borgosatollo
Genemuiden
Derby
Toulouse
Paris
Ljubljana
Aix-en-Provence
Carugate
Stroud
Sheffield
Ozzano dell'Emilia
Strassen
Beveren
Split
Genoa
Bucharest
Helsinki
Lugano
Amsterdam
Rome
Espoo
Birmingham
Oslo
Moscow
Șofronea
Milan
Ennepetal
Munich
Poitiers
Vicenza
Leuven
Bærum
Ballerup
Valverde del Majano
Almagro
Kristiansund
George Town
Sola
Singapore
Glenrothes
Las Vegas
Woodstock
Houten
Iona
Cologne
Robbio
Utrecht
Poznań
Origgio
Glasgow
Douglas
Geneva
São Paulo
Dallas
Elkhart
Vienna
Leawood
Portland
Pedrengo
Sacro Cuore
Deinze
Madrid
Nuremberg
Novosibirsk
Saint-Bruno-de-Montarville
Frankfurt
Lausanne
Sydney
Pancevo City
Reykjavík
Dundee
Ba

Network for Founders

In [13]:
founderJsonExtracted=jsonListTransformerFounder(founders_json)

In [14]:
with open("Founders_clean.json", "w") as fp:
    json.dump(founderJsonExtracted, fp) 

In [15]:
import gzip
with gzip.open("Founders_clean.json.gz", 'wt', encoding='utf-8') as f:
    json.dump(founderJsonExtracted, f)


In [33]:
g_founders=nx.Graph()

In [34]:
for founder in founderJsonExtracted:
    attr=founder.copy()
    node_name=attr.pop("id")
    g_founders.add_node(node_name,**attr)

In [35]:
city_founders_dict=defaultdict(list)
for node,data in g_founders.nodes(data=True):
    cities=data.get("city")
    for city in cities:
        if city is not None:
            city_founders_dict[city].append(node)

for key, values in city_founders_dict.items():
    print(key)
    input=list(itertools.product(values,values))
    seen = set()
    # using list comprehension
    Output = [(a, b, 2) for a, b in input if not (a == b) ]
    g_founders.add_weighted_edges_from(Output, edge_type="city", city=key, edge_color="red")

Zurich
Copenhagen
Aarau
Dornbirn
Olten
Paris
Verona
Hausen am Albis
Geneva
Novosibirsk
Lisbon
Leiria
Cittadella
Hemberg
Uster
Dübendorf
Basel
Sandy
Málaga
Beetsterzwaag
Zoetermeer
Bordeaux
Barcelona
Alcobendas
St. Gallen
Illnau-Effretikon
Fontenay-sous-Bois
Contern
Girona
Bern
Madrid
Helsinki
Chesham Bois
Meerssen
Dublin
Porto
Oviedo
Nuremberg
Stockholms kommun
Moscow
Chisinau
Prague
Göteborgs Stad
Sundsvall Municipality
Constanța
Vienna
Turku
Umeå kommun
Lappeenranta
Jyväskylä
Sliema
Espoo
's-Hertogenbosch
Nivå
Plzeň
Oulu
Karlstads kommun
Limassol Municipality
Oslo
Linköpings kommun
Vellinge
Belgrade
Peterborough
Uppsala kommun
Rheinfelden
Dortmund
Innsbruck
Pori
Halmstads kommun
Aalborg
Budapest
Helsingborgs kommun
Lund Municipality
Vilnius
Tampere
Tallinn
Måløv
Biarritz
Zaslawye
Luleå kommun
Salzbergen
Groß-Bieberau
Aulendorf
Eura
Unterach am Attersee
Graz
Vesanto
Tirana Municipally
Mölndals kommun
Lugano
Košice
Athens
Södertälje kommun
Viterbo
Warsaw
Täby
Tilburg
Minsk
Leeuwarden
D

: 

: 

In [ ]:
investor founder connection

In [ ]:
for founder in founderJsonExtracted:
    for investor in investorJsonExtracted:
        if founder["investor"] in investor["linkedin_url"]:
            g_founders.add_edge(founder["name"],investor["name"],edge_type="investor",edge_color="blue")

Network Analysis

Connections for Investors
- Same city (maybe country)
- Same investments
- (Same Investment focus / Industry)
- Background of Investors

Connections for Universities
- no direct connections (at least not visible in the data) (Partner Universities)
- alumni 
- Investors

Connections for Companies
- alumni
- same investors

Connections for Founders
- Same city 
- Same University
- Same Background



Network for Investors

Useful links:
- https://www.datacamp.com/tutorial/social-network-analysis-python


Attributes important as Investor
- industry experience
- country experience
- total funding source
- investments num
- top investments num
- fundings total
- co inv total

In [13]:
import networkx as nx

In [125]:
df_investors["investor_type"].unique()

array(['government & non-profit', 'corporate', 'company',
       'investment fund', 'angel', 'crowdfunding', 'service provider',
       'university', 'workspace', 'user', 'event', 'workspace location'],
      dtype=object)

In [30]:
types_to_exclude = ["crowdfunding", "company", "government & non-profit"]
df_investors = df_investors.loc[~df_investors["investor_type"].isin(types_to_exclude) & (df_investors["total_funding"] > 0)]

In [23]:
g_inv=nx.Graph()

In [14]:
for index, row in df_investors.iterrows():
    g_inv.add_node(row["name"],network_type="Investors", node_type="Fund/Investment Vehicle", id=row["id"],industry_experience=row["industry_experience"],country_experience=row["country_experience"],total_funding_source=row["total_funding_source"],investments_num=row["investments_num"],top_investments_num=row["top_investments_num"],fundings_total=row["fundings_total"],co_investors_total=row["co_investors_total"]) 

NameError: name 'g_inv' is not defined

City Connection

In [18]:
cities=[]
for i in df_investors["city"]:
    cities.extend(i)
    cities=pd.unique(cities).tolist()
cities=[a for a in cities if str(a) != 'nan']

In [19]:
city_inv_dict={}
for i in cities:
    city_inv_dict[i]=df_investors[df_investors["city"].apply(lambda x: i in x)]["name"].tolist()

In [21]:
import itertools
t1 = ['A', 'B']
t2 = ['C', 'D', 'E']
list(itertools.product(t1, t2))


[('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('B', 'E')]

In [25]:
for key, values in city_inv_dict.items():
    print(key)
    input=list(itertools.product(values,values))
    seen = set()
    # using list comprehension
    Output = [(a, b, 2) for a, b in input if not (a == b) ]
    g_inv.add_weighted_edges_from(Output, edge_type="city", city=key, edge_color="red")

Wilmington
Brno
Windhoek
Split
Bucharest
Amsterdam
Paris
Birmingham
Oslo
Milan
Ennepetal
Moscow
Poitiers
Leuven
Bærum
Valverde del Majano
Almagro
George Town
Sola
Singapore
Woodstock
Iona
Robbio
Apeldoorn
Barcelona
Origgio
Bern
Sint-Lambrechts-Woluwe
Berlin
Brussels
London
Athens
Richmond
Munich
Umeå kommun
São Paulo
Leeds
Dallas
Portland
Deinze
Madrid
Shanghai
Nuremberg
Stockholms kommun
Genoa
Sydney
Reykjavík
Tel Aviv-Yafo
Los Angeles
Bangkok
Mountain View
Kyiv
Beaverton
Huntsville
Oñati
Bad Wiessee
Monterrey
Pont-à-Celles
Fremont
Lausanne
Toronto
Barry
Arnac-Pompadour
Dublin
Chicago
Duluth
Richmond Hill
Tokyo
Uppsala kommun
Vihti
Ulm
Mumbai
Kempton Park
Ahmedabad
Rennes
L'Aquila
Winnetka
Helsinki
Jeddah
Heidelberg
Aalborg
Nyborg
Knoxville
Hamburg
Plock
Kyōbashi
Komaki-Shi
North Kingstown
Vantaa
Dubai
Zurich
Beijing
Oulu
Rouen
Utrecht
Eindhoven
Downtown Core
Ljubljana
Harrisburg
Region of Murcia
Edinburgh
Burnaby
Gräfelfing
Warrington
Wissous
Trento
Dunfermline
Gijón/Xixón
Austin
San

Prev. Investments

In [4]:
investments={}
for i in df_investors["investments_items"]:
    for key,value in i.items():
        if str(value)!= "nan":
            investments[key]=value



In [32]:
investors={}
for index,row in df_investors.iterrows():
    if str(row["name"])!= "nan":
        investors[row["id"]]=row["name"]

In [33]:
co_investors={}
for i in df_investors["co_investors_items"]:
    for key,value in i.items():
        if str(value)!= "nan":
            co_investors[key]=value

In [129]:
missing_investors={}
for key,values in co_investors.items():
    if key not in investors.keys():
        missing_investors[key]=values

In [18]:
investments_dict={}
for i in investments:
    investments_dict[i]=df_investors[df_investors["investments_items"].apply(lambda x: i in x)]["name"].tolist()

In [19]:
co_investors=[]
investors_list=[]
for key, values in investments_dict.items():
    co_investors.append(values)
    investors_list.extend(values)

In [39]:
investor_co_dict={}
for i in investors_list:
    for j in co_investors:
        if i in j:
            investor_co_dict[i]=[]
            investor_co_dict[i].extend(j)
    investor_co_dict[i]=pd.unique(investor_co_dict[i]).tolist()

In [54]:
missing_investors_info=missing_info(list(missing_investors.keys()),"investors")

In [40]:
co_inv_dict=dict(zip((df_investors["name"].tolist()),(df_investors["co_investors_items"].tolist())))

In [41]:
for key, values in investor_co_dict.items():
    try:
        co_inv_dict[key].extend(values)
        co_inv_dict[key]=pd.unique(co_inv_dict[key]).tolist()
    except:
        co_inv_dict[key]=values

In [47]:
for key, values in co_inv_dict.items():
    print(key)
    input=list(itertools.product(values,values))
    seen = set()
    # using list comprehension
    Output = [(a, b, 5) for a, b in input if not a == b]
    g_inv.add_weighted_edges_from(Output, edge_type="co-Investors", investment=key, edge_color="blue")

Clean Earth Acquisitions Corp
Voutein holding
Club2021
Far Point Acquisition Corporation
LavasDesign
Garage Angels
Moyreal Holding
Styrian Economic Development Agency
Angel
EMW Consulting
MultiChain Advisors
Grupo Virto
Jadroplov
EGV International
venturecake
Good Tiding Holding
SAIC
Volksbank Raiffeisenbank Pressath
Premius
Spinnaker Acquisitions
USM Global Corporation
Seagrow
Syner
Finest
CF Capital Partners
Americorp Financial
Penny Venture
Climate Collective
WeChange group
Vater
BIC
Med Group
Grupo
Aussie Capital
MCG Technology Group
Gruppo Brixia
revo-spac
Bilstein Group
TICCYL Digital
RosGeo
Goldmann International
Gramtec Co-Invest
Esibiusi
Sorégies
SBB Accountants & Advisors
Metacollective
Pop Invest
Metplanet Holdings
Tschudi Tank Transport
Brandenburg State Investment Bank
Ceyde
CRB Bio
Centricus Acquisition Corp
NorSea Group
FTAG
Glatthaar Holding
George Shultz Innovation Fund
MMC
Pack Capital
Fortuna
Yuzhnaya Torgovaya Kompaniya
Woodstock Financial
Privcode Capital
Inclusion

In [49]:
with open("g_inv.pkl", 'wb') as f:
    pickle.dump(g_inv, f)

In [52]:
centrality = nx.degree_centrality(g_inv)
node_centrality = centrality['Volksbank Raiffeisenbank Pressath']


First we have to prepare the dataset such that connections become clear

Graph Founders

Needed Connections:
- City
- University
- Background


In [5]:
with open("/Users/klaus/Documents/Jarvis/Second Try/app/model/Inv./Eurpe_Founders.json", 'r') as fp:
    founders_json = json.load(fp)
    


df_list=[]
for i in range(len(founders_json)):
        try:
                df_1=pd.json_normalize(founders_json[i]["items"],sep="_")
                df_list.append(df_1)
        except: None

df_founders=pd.concat(df_list)

In [9]:
sum([False,True,False])

1

In [6]:
df_founders=df_founders[["id","name","backgrounds","linkedin_url","hq_locations","universities_items","companies_items","founder_score",'is_strong_founder', 'is_super_founder', 'is_promising_founder', 'founded_companies_total_funding','universities_total','companies_total']]

In [32]:
def jsonListTransformerFounder(apiOutput: list):
  networkInputList=[]
  for singleJsonList in apiOutput: 
    jsonList=singleJsonList["items"]
    for individualJson in jsonList:  
      if  individualJson["linkedin_url"] is not None:     
          networkInput={}
          networkInput["id"]=individualJson["id"]
          networkInput["type"]=individualJson["type"]
          networkInput["name"]=individualJson["name"]
          networkInput["backgrounds"]=[x["name"] for x in individualJson["backgrounds"] if "name" in x.keys()]
          networkInput["linkedin_url"]=individualJson["linkedin_url"]
          networkInput["hq_locations"]=[x["city"]["name"] for x in individualJson["hq_locations"] if "city" in x.keys() and x["city"] is not None]
          networkInput["universities"]=[{x["university"]["id"] :x["university"]["name"]} for x in individualJson["universities"]["items"] if "university" in x.keys()]
          networkInput["universities_degree"]=[x["degree"]["name"] for x in individualJson["universities"]["items"] if "degree" in x.keys() and x["degree"] is not None]
          networkInput["companies"]=[{x["id"]:x["name"]} for x in individualJson["companies"]["items"] if "id" in x.keys()]
          networkInput["companies_titles"]=[sublist["name"] for x in individualJson["companies"]["items"] for sublist in x["titles"] if "titles" in x.keys() and x["titles"]!=[]]
          networkInput["founded_companies"]=sum([x["is_founder"] for x in individualJson["companies"]["items"] if "is_founder" in x.keys()])
          networkInput["founder_score"]=individualJson["founder_score"]
          networkInput["is_strong_founder"]=individualJson["is_strong_founder"]
          networkInput["is_super_founder"]=individualJson["is_super_founder"]
          networkInput["is_promising_founder"]=individualJson["is_promising_founder"]
          networkInput["founded_companies_total_funding"]=individualJson["founded_companies_total_funding"]
          networkInput["universities_total"]=individualJson["universities"]["total"]
          networkInput["companies_total"]=individualJson["companies"]["total"]
          networkInputList.append(networkInput)
      else: None
    return networkInputList

In [35]:
founderJsonExtracted=jsonListTransformerFounder(founders_json)

In [7]:
df_founders["companies_items"]=df_founders["companies_items"].apply(lambda x: level1_extractor_dict(x,"id","name"))
df_founders["universities_items"]=df_founders["universities_items"].apply(lambda x: level1_extractor_dict(x,"id","name"))
df_founders["city"]=df_founders["hq_locations"].apply(lambda x: level1_extractor_list(x,"city.name"))

In [18]:
df_founders.shape

(346947, 15)

In [8]:
background_companies={}
for i in df_founders["companies_items"]:
    for key,value in i.items():
        if str(value)!= "nan":
            background_companies[key]=value


In [10]:
len(background_companies)

237332

In [10]:
company_dict=dict(zip(df_countries["id"].tolist(),df_countries["name"].tolist()))

In [11]:
missing_companies={}
for key,value in background_companies.items():
    if key not in company_dict.keys():
        missing_companies[key]=value
        

In [39]:
len(missing_companies)

147626

In [12]:
missing_companies_inv={}
for key,value in investments.items():
    if key not in company_dict.keys():
        missing_companies_inv[key]=value

In [40]:
len(missing_companies_inv)

47325

In [13]:
missing_companies.update(missing_companies_inv)


In [14]:
missing_companies_1={}

for key,value in missing_companies.items():
    if key not in missing_companies_1.keys():
        missing_companies_1[key]=value

In [15]:
missing_companies_list=list(missing_companies_1.keys())

In [157]:
len(missing_companies_list)

189422

In [163]:
chunks=[missing_companies_list[x:x+50] for x in range(0, len(missing_companies_list), 50)]


In [164]:
def id_getter(chunk):
    id_str=""
    for i in chunk:
        id_str=id_str+str(i)+","
    id_str=id_str[:-1]
    api_url="https://api.dealroom.co/api/v1/companies/batch?ids="+id_str+"&fields="+fields
    auth=HTTPBasicAuth(API_KEY, '')
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Basic undefined"
    }

    response = requests.get(url=api_url, auth=auth, headers=headers)

    res=json.loads(response.text)
    return res

In [168]:
import re
l=[]
failed=[]
k=0
for chunk in chunks:
    try:
        res=id_getter(chunk)
        if "error" in res.keys():
            a=re.findall(r'\d+', res["message"])
            for i in a:
                chunk.remove(int(i))
                l.append(id_getter(chunk))
        else:
            l.append(res)
    except: failed.extend(chunk)
    k+=1
    print(str(k)+"/"+str(len(missing_companies_list)/50),end="\r")

In [169]:
len(failed)

50

In [96]:
len(failed)

1000

In [172]:
savin_dashit=l

In [174]:
failed[0]

227697

In [175]:
id_str=""
k=0
for i in failed:    
    try:
        api_url="https://api.dealroom.co/api/v1/companies/batch?ids="+str(i)+"&fields="+fields
        auth=HTTPBasicAuth(API_KEY, '')
        headers = {
            "Content-Type": "application/json",
            "Authorization": "Basic undefined"
        }

        response = requests.get(url=api_url, auth=auth, headers=headers)

        res=json.loads(response.text)
        l.append(res)
        k+=1
        print(k,end="\r")
    except: print(str(id)+" failed")

<built-in function id> failed
<built-in function id> failed


In [129]:
"error" in res.keys()

True

In [130]:
import re
a=re.findall(r'\d+', res["message"])
for i in a:
    print(i)

3884977


In [98]:
chunk.remove(3884977)

In [94]:
3884977 in chunks[0]

True

In [75]:
"error" in res.keys()

True

In [78]:
res

{'error': True, 'code': 404, 'message': 'No item with id 0 was found.'}

In [156]:
len(failed)

3000

In [176]:
len(l)

3847

In [177]:
df_list=[]
for i in range(len(l)):
        try:
                df_1=pd.json_normalize(l[i]["items"],sep="_")
                df_list.append(df_1)
        except: None

df_missing_comp=pd.concat(df_list)

In [178]:
df_missing_comp.shape

(189135, 28)

In [179]:
df_missing_comp.to_pickle("Missing_Companies.pkl")

In [57]:
df_missing_comp=pd.read_pickle("Missing_Companies.pkl")

In [19]:
id_str

'927887,935537,1655362,3754375,79402,908366,194975,3268506,983625,1704296,3952004,1990009,3871747,1251899,3425869,3058317,16161,4284099,4609979,935077,3491361,1512610,935111,1287340,1472781,3118002,4472089,1887250,2390304,865722,2731698,3054239,1758482,909697,163410,2024766,4377166,935724,967420,1999524,3066355,3428892,913444,2931091,3884977,1831753,1711041,4287224,83304,3524892,4611182,1297738,3428934,973656,3912496,3246384,1891250,3990040,1888353,4613073,144607,3154693,2946751,901832,1347405,1406411,3349330,2935787,1467902,935215,4464122,3093333,226155,2778567,1835626,111480,3045673,3214985,1383461,1681458,4610240,974211,4610254,936003,3075504,4284108,4534911,2267783,28152,895949,16872,1353660,2885352,1455861,143674,1762949,2021601,954198,1419325,3120317,935465,1565443,1261472,3299678,1708512,3446573,3531195,3738336,4413480,936016,4655350,394826,873525,4141026,907113,2518,4643154,1678,2159264,3564291,3570524,3564293,4615235,935899,4375212,936282,3564349,4628718,3564804,3752447,307325

In [182]:
df_missing_comp.shape

(189135, 28)

In [189]:
[a for a in df_missing_comp.columns.to_list() if a not in df_countries.columns.to_list()]

['industries',
 'hq_locations',
 'dealroom_signal_rating',
 'dealroom_signal_completeness',
 'dealroom_signal_team_strength',
 'dealroom_signal_growth_rate',
 'dealroom_signal_timing']

In [195]:
df_missing_comp.head()

,id,name,about,tagline,industries,website_url,facebook_url,twitter_url,linkedin_url,hq_locations,total_funding,company_status,ownerships,patents_count,employees_latest,employees_chart,launch_year,investors_items,investors_total,team_items,team_total,fundings_items,fundings_total,dealroom_signal_rating,dealroom_signal_completeness,dealroom_signal_team_strength,dealroom_signal_growth_rate,dealroom_signal_timing
0,927887,Essec,100 years of excellence and innovation \n\nFor...,None,[],https://www.essec.edu/fr/,None,https://twitter.com/essec,https://www.linkedin.com/company/essec-busines...,"[{'id': 3142334, 'address': 'Cergy, Pontoise, ...",0.00,None,[],0,1961.0,"[{'date': '2021-12-21', 'value': 1961}]",NaN,[],0,"[{'id': 4197841, 'name': 'Yassine CHARAI', 'pa...",50,[],0,NaN,NaN,NaN,NaN,NaN
1,935537,Eidgenössische Technische Hochschule Zürich (E...,None,"Switzerland-based engineering, science, techno...",[],https://www.ethz.ch/,None,https://twitter.com/eth,https://www.linkedin.com/school/eth-zurich/,"[{'id': 750718, 'address': '101 Rämistrasse, 8...",0.00,None,[],0,NaN,[],NaN,[],0,"[{'id': 1861029, 'name': 'Ferdinand von Hagen'...",920,[],0,NaN,NaN,NaN,NaN,NaN
2,1655362,EMPA,Empa is the interdisciplinary research institu...,Empa is the interdisciplinary research institu...,[],https://www.empa.ch,None,https://twitter.com/empa_ch,https://www.linkedin.com/company/empa/,"[{'id': 2644573, 'address': 'Switzerland', 'st...",0.00,None,[],0,NaN,[],NaN,[],0,"[{'id': 1074647, 'name': 'Bastjan Prenaj', 'pa...",43,[],0,NaN,NaN,NaN,NaN,NaN
3,3754375,DuraMon,None,None,"[{'id': 1270, 'name': 'real estate'}]",https://duramon.ch,None,None,https://www.linkedin.com/company/duramon/,"[{'id': 4304704, 'address': 'Laura-Hezner-Weg,...",0.83,operational,[Venture Capital],0,NaN,[],NaN,"[{'id': 935537, 'name': 'Eidgenössische Techni...",6,"[{'id': 4611974, 'name': 'Yurena Seguí Femenia...",3,"[{'id': 791256, 'year': 2023, 'month': 6, 'amo...",2,17.0,15.75,0.0,0.0,14.75
4,79402,Clipperton Finance,None,Investment banking boutique - TMT Europe,[],http://clipperton.net,None,https://twitter.com/clippertonfin,https://www.linkedin.com/company/clipperton-fi...,"[{'id': 26957, 'address': '10, Rue du Mont Tha...",0.00,operational,[Private Equity],0,39.0,"[{'date': '2019-04-01', 'value': 28}, {'date':...",2007.0,[],0,"[{'id': 867092, 'name': 'Stephane Valorge', 'p...",38,[],0,NaN,NaN,NaN,NaN,NaN


In [194]:
df_missing_comp.applymap(lambda x: np.nan if x==[] else x)

KeyboardInterrupt: 

In [190]:
(df_missing_comp.isnull().sum()/df_missing_comp.shape[0]).sort_values(ascending=False) * 100

about                            85.824940
facebook_url                     62.262405
dealroom_signal_timing           54.734449
dealroom_signal_rating           54.734449
dealroom_signal_growth_rate      54.734449
dealroom_signal_team_strength    54.734449
dealroom_signal_completeness     54.734449
twitter_url                      39.743569
tagline                          28.753007
employees_latest                 18.667090
linkedin_url                     13.878447
launch_year                      10.711925
company_status                    3.560420
website_url                       1.135168
total_funding                     0.000000
team_total                        0.000000
industries                        0.000000
hq_locations                      0.000000
fundings_total                    0.000000
fundings_items                    0.000000
team_items                        0.000000
investors_total                   0.000000
investors_items                   0.000000
employees_c

In [181]:
df_missing_comp.loc[~df_missing_comp["id"].isin(df_countries["id"].tolist())].shape

(189135, 28)

In [16]:
fields="id,name,about,tagline,investors,industries,website_url,facebook_url,twitter_url,linkedin_url,hq_locations,total_funding,company_status,ownerships,team,fundings,patents_count,employees_latest,employees_chart,launch_year,dealroom_signal"

In [20]:
api_url="https://api.dealroom.co/api/v1/companies/batch?ids="+id_str+"&fields="+fields
auth=HTTPBasicAuth(API_KEY, '')
headers = {
    "Content-Type": "application/json",
    "Authorization": "Basic undefined"
}

response = requests.get(url=api_url, auth=auth, headers=headers)

res=json.loads(response.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
l=[]
k=0
for i in failed: 
    k+=1
    print(k,end="\r")
    try:
        api_url="https://api.dealroom.co/api/v1/investors/batch?ids="+str(i)+"&fields="+fields
        auth=HTTPBasicAuth(API_KEY, '')
        headers = {
            "Content-Type": "application/json",
            "Authorization": "Basic undefined"
        }

        response = requests.get(url=api_url, auth=auth, headers=headers)

        res=json.loads(response.text)
        l.append(res)
    
    except: None
    


In [ ]:
with open("mssing_companies.json", "w") as fp:
    json.dump(l, fp) 

Graph Companies

In [9]:
df_countries=pd.read_pickle("/Users/klaus/Documents/Jarvis/General Fcts./Second Try Rest/rest/df_new(countries).pkl")

In [57]:
df_countryNames=df_countries["name"].tolist()
a=[a for a in investments if a not in df_countryNames]